# Imports

In [738]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [739]:
torch.manual_seed(1337)
training_style = "nscale"
if training_style == "scale":
    block_size = 32
    batch_size = 512
    split_pct = 0.9
    max_iters = 5000
    eval_interval = 500
    learning_rate = 3e-4 #1e-3
    eval_iters = 200
    n_embd = 384 #64
    n_layer = 6 # Number of block layers
    n_head = 6
    dropout = 0.2
else:
    block_size = 32
    batch_size = 256
    split_pct = 0.9
    max_iters = 500
    eval_interval = 100
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 64
    n_layer = 2 # Number of block layers
    n_head = 2
    dropout = 0.2

In [740]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

mps


# Dataloader

In [741]:
chars = ['\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '.', '=', 'e']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 16


In [742]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] if c in chars else print (s) for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [743]:
def get_batch(batch_size):
    numbers = torch.randint(0, 1000000, (batch_size+4, 2))
    data = ""
    ix = []
    for idx in range(numbers.size(0)):
        eqn = "{}+{}={}".format(numbers[idx, 0], numbers[idx, 1], numbers[idx, 0]+numbers[idx, 1])
        if idx > 2 and idx < (numbers.size()[0] - 1):
            ix.append(torch.randint(len(data) + eqn.index("=") + 2, len(data) + len(eqn) + 1, (1,)).item())
        data = data + "\\" + eqn
    data = torch.tensor(encode(data))
    x = torch.stack([data[i-block_size:i] for i in ix])
    # y here is the target - the next token for any given point in x
    y = torch.stack([data[i-block_size+1:i+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [744]:
x, y = get_batch(batch_size)
x.size(), y.size()

(torch.Size([256, 32]), torch.Size([256, 32]))

In [745]:
for idx in range(0, len(x), 50):
    print(decode(x[idx].tolist()), decode(y[idx].tolist()))

+513753=1058048\751015+906010=16 513753=1058048\751015+906010=165
2989=1563524\753748+689526=14432 989=1563524\753748+689526=144327
+740049=1087731\994298+696814=16 740049=1087731\994298+696814=169
747+791227=1135974\94533+784841= 47+791227=1135974\94533+784841=8
08025=873432\468402+179764=64816 8025=873432\468402+179764=648166
6196=654701\650051+753387=140343 196=654701\650051+753387=1403438


# Model

In [746]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch(batch_size)
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [747]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [748]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [749]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [750]:
class GptModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=encode("\\")[0])
            
        return logits, loss

    def generate(self, no_sums):
        # idx is (B, T) array of indices in the current context
        x, y = get_batch(no_sums)
        max_new_tokens = 12
        results = []
        for idx, eqn in enumerate(x):
            for _ in range(max_new_tokens):
                # crop idx to last block_size tokens
                eqn_cond = eqn[:, -block_size:]
                # get predictions
                logits, loss = self(eqn_cond)
                # focus only on last time step
                logits = logits[:, -1, :]
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # )B, C)
                # sample from distribution
                eqn_next = torch.multinomial(probs, num_samples=1) # (B, 1)
                # append sampled index to the running sequence
                eqn = torch.cat((eqn, eqn_next), dim=1) # (B, T+1)
            results.append((decode(eqn[0].tolist())), decode(y[idx].tolist()))
        return results

In [751]:
m = GptModel()
m.to(device)

GptModel(
  (token_embedding_table): Embedding(16, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)

In [752]:
# Number of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.103824 M parameters


# Training

In [753]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [759]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch(batch_size)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.0655, val loss 2.0664
step 100: train loss 2.0637, val loss 2.0625
step 200: train loss 2.0580, val loss 2.0579
step 300: train loss 2.0538, val loss 2.0536
step 400: train loss 2.0500, val loss 2.0511
step 499: train loss 2.0483, val loss 2.0481


# Inference

In [755]:
def gen_sums(num):
    numbers = torch.randint(0, 1000000, (num, 2))
    x = []
    y = []
    for ix in range(num):
        # Create equation with solution digits reversed
        eqn = "{}+{}=".format(numbers[ix,0], numbers[ix,1], str((numbers[ix,0] + numbers[ix,1]).item()))
        answ = (numbers[ix,0] + numbers[ix,1]).item()
        # Add padding so input and target both have lengths equal to block size
        x.append(torch.tensor(encode("\\" * (block_size - len(eqn)) + eqn)))
        y.append(answ)
    return torch.stack(x).to(device), y

In [756]:
def generate(no_sums):
    # idx is (B, T) array of indices in the current context
    x, answer = gen_sums(no_sums)
    max_new_tokens = 7
    results = []
    for idx, eqn in enumerate(x):
        eqn = eqn.unsqueeze(0)
        for _ in range(max_new_tokens):
            # crop idx to last block_size tokens
            eqn_cond = eqn[:,-block_size:]
            # get predictions
            logits, loss = m(eqn_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # )B, C)
            # sample from distribution
            eqn_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            eqn = torch.cat((eqn, eqn_next), dim=1) # (B, T+1)
        results.append((decode(eqn[0].tolist()).replace("\\",""), answer[idx]))
    return results

In [757]:
inference = generate(no_sums=10)
for calc in inference:
    print("Prediction: {}\nReal Result:{}".format(calc[0], str(calc[1])[::-1]))

Prediction: 676527+697957=6249137
Real Result:4844731
Prediction: 86682+290305=8151931
Real Result:789673
Prediction: 140266+677220=7353189
Real Result:684718
Prediction: 339908+70766=3981485
Real Result:476014
Prediction: 125786+834088=9188137
Real Result:478959
Prediction: 447840+73942=9596239
Real Result:287125
Prediction: 737089+531394=7116931
Real Result:3848621
Prediction: 174774+591310=8112512
Real Result:480667
Prediction: 854391+576160=6391915
Real Result:1550341
Prediction: 384523+888484=9291121
Real Result:7003721


# Save the model

In [758]:
#import time
#torch.save(m.state_dict(), './models/addition_calculator_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))